### 練習問題2.71
$n$記号のアルファベットに対するハフマン木があり、  
記号の相対頻度は$1,2,4,...,2^{n−1}$であるとする。  
$n = 5$、$n = 10$の場合の⽊をスケッチせよ。  
そのような木では、(⼀般の$n$について )最も頻度の⾼い記号を符号化するのに何ビット必要になるだろうか。  
最も頻度の低い記号はどうだろうか。

木の回答は保留にする。

In [1]:
(define (make-leaf symbol weight) (list 'leaf symbol weight))
(define (leaf? object) (eq? (car object) 'leaf))
(define (symbol-leaf x) (cadr x))
(define (weight-leaf x) (caddr x))

; コンストラクタ
(define (make-code-tree left right)
  (list left right
        (append (symbols left) (symbols right))
        (+ (weight left) (weight right))
    )
  )

; セレクタ
(define (left-branch tree) (car tree))
(define (right-branch tree) (cadr tree))
(define (symbols tree)
  (if (leaf? tree) (list (symbol-leaf tree))
      (caddr tree)
    )
  )
(define (weight tree)
  (if (leaf? tree) (weight-leaf tree)
      (cadddr tree)
    )
  )

(define (decode bits tree)
  (define (decode-1 bits current-branch)
    (if (null? bits) '()
        (let ((next-branch (choose-branch (car bits) current-branch)))
          (if (leaf? next-branch) (cons (symbol-leaf next-branch) (decode-1 (cdr bits) tree))
              (decode-1 (cdr bits) next-branch))
          )
    )
    )
  (decode-1 bits tree)
)  
(define (choose-branch bit branch)
  (cond ((= bit 0) (left-branch branch))
        ((= bit 1) (right-branch branch))
        (else (error "bad bit: CHOOSE-BRANCH" bit))
        )
  )

(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((< (weight x) (weight (car set))) (cons x set))
        (else (cons (car set) (adjoin-set x (cdr set))))))

(define (make-leaf-set pairs)
  (if (null? pairs) '()
      (let ((pair (car pairs)))
        (adjoin-set (make-leaf (car pair) ; symbol
                               (cadr pair)) ; weight
                    (make-leaf-set (cdr pairs)))
        )
    )
  )

In [2]:
; 符号化処理
(define (encode-symbol symbol tree)
  (define (iter sub result)
    (if (leaf? sub)
        (if (eq? (symbol-leaf sub) symbol) result
            '())
        (let ((l (left-branch sub))
              (r (right-branch sub)))
          (let ((l-result (iter l (append result '(0)))))
            (if (not (null? l-result)) l-result
                (iter r (append result '(1)))
              )
            )
        )
      )
    )
  (let ((result (iter tree '())))
    (if (null? result) (error "bad symbol: ENCODE" symbol)
        result
        )
    )
)

; 符号化処理 
(define (encode message tree)
    (if (null? message) '()
        (append
            (encode-symbol (car message) tree)
            (encode (cdr message) tree))
    )
)

In [3]:
; 考え直した回答

; 指定した頻度がリストに含まれているか。
; 含まれていない場合、falseを返す。
; 含まれている場合、最初に見つかったペアと次のペアを返す。（これの2つで木を生成する）
(define (have-weight? w set)
  ;(display "have-weight? ")
  ;(display w)
  ;(display " ")
  ;(display set)
  ;(display " ")
  ;(newline)
  (cond ((null? set) #f)
        ((not (pair? set))
         (if (= (weight set) w) (list set)
             #f
             )
         )
        (else
         (if (= (weight (car set)) w) (list (car set) (cadr set))
             (have-weight? w (cdr set))
             )
         )
    )
)

; ハフマン符号木の生成の記号・頻度ペアのリストから、指定したペアを削除したリストを返す。
(define (remove-info item set)
  (if (null? set) '()
      (if (equal? item (car set)) (remove-info item (cdr set))
          (cons (car set) (remove-info item (cdr set)))
      )
  )
  )

; 回答
(define (successive-merge set)
  (define (iter w result)
    ;(display result)
    ;(newline)
    (cond ((null? result) '())
          ((= (length result) 1) (car result))
          ;((not (pair? result)) result) ; これは動作しない
          ((= (length result) 2) (make-code-tree (cadr result) (car result)))
          (else
           (let ((ll (have-weight? w result)))
             (if (equal? ll #f) (iter (+ w 1) result)
                 (let ((top (car ll))
                       (next (cadr ll)))
                   (let ((new-result (remove-info next (remove-info top result))))
                     (if (null? new-result) (iter w (list (make-code-tree next top)))
                         (if (= (length new-result) 1) (iter w (append (list (make-code-tree next top)) new-result))
                             (let (;(new-item (make-code-tree next top))
                                   (new-item (if (>= (weight next) (weight top)) (make-code-tree next top) (make-code-tree top next)))
                                   )
                               (if (<= (weight new-item) (weight (car new-result)))
                                   (iter w (append (list new-item) new-result))
                                   (iter w (append new-result (list new-item)))
                                )
                              )
                           )
                        )
                     )
                    )
                )
             )
           )
        )
    )
  (iter 1 set)
  )

(define (generate-huffman-tree pairs)
    (successive-merge (make-leaf-set pairs)))

In [4]:
; 2^(5-1)
(define pairs1 '((A 16) (B 8) (C 4) (D 2) (E 1)))
(define sample1 (generate-huffman-tree pairs1))
sample1

((leaf A 16) ((leaf B 8) ((leaf C 4) ((leaf D 2) (leaf E 1) (D E) 3) (C D E) 7) (B C D E) 15) (A B C D E) 31)

In [5]:
; 単一の記号で動作確認
(display (encode '(A) sample1))
(newline)
(display (encode '(B) sample1))
(newline)
(display (encode '(C) sample1))
(newline)
(display (encode '(D) sample1))
(newline)
(display (encode '(E) sample1))
(newline)

(0)
(1 0)
(1 1 0)
(1 1 1 0)
(1 1 1 1)


In [6]:
; 2^(10-1)
(define pairs2 '((A 512) (B 256) (C 128) (D 64) (E 32) (F 16) (G 8) (H 4) (I 2) (J 1)))
(define sample2 (generate-huffman-tree pairs2))
sample2

((leaf A 512) ((leaf B 256) ((leaf C 128) ((leaf D 64) ((leaf E 32) ((leaf F 16) ((leaf G 8) ((leaf H 4) ((leaf I 2) (leaf J 1) (I J) 3) (H I J) 7) (G H I J) 15) (F G H I J) 31) (E F G H I J) 63) (D E F G H I J) 127) (C D E F G H I J) 255) (B C D E F G H I J) 511) (A B C D E F G H I J) 1023)

In [7]:
; 単一の記号で動作確認
(display (encode '(A) sample2))
(newline)
(display (encode '(B) sample2))
(newline)
(display (encode '(C) sample2))
(newline)
(display (encode '(D) sample2))
(newline)
(display (encode '(E) sample2))
(newline)
(display (encode '(F) sample2))
(newline)
(display (encode '(G) sample2))
(newline)
(display (encode '(H) sample2))
(newline)
(display (encode '(I) sample2))
(newline)
(display (encode '(J) sample2))
(newline)

(0)
(1 0)
(1 1 0)
(1 1 1 0)
(1 1 1 1 0)
(1 1 1 1 1 0)
(1 1 1 1 1 1 0)
(1 1 1 1 1 1 1 0)
(1 1 1 1 1 1 1 1 0)
(1 1 1 1 1 1 1 1 1)
